In [4]:
import pandas as pd

# Try reading the CSV file with different encodings
try:
    df = pd.read_csv('C:/Users/Hi/Downloads/archive (3)/IMDb Movies India.csv', encoding='latin-1')
    # Proceed with data analysis or preprocessing
    print(df.head())
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")


                                 Name    Year Duration            Genre  \
0                                         NaN      NaN            Drama   
1  #Gadhvi (He thought he was Gandhi)  (2019)  109 min            Drama   
2                         #Homecoming  (2021)   90 min   Drama, Musical   
3                             #Yaaram  (2019)  110 min  Comedy, Romance   
4                   ...And Once Again  (2010)  105 min            Drama   

   Rating Votes            Director       Actor 1             Actor 2  \
0     NaN   NaN       J.S. Randhawa      Manmauji              Birbal   
1     7.0     8       Gaurav Bakshi  Rasika Dugal      Vivek Ghamande   
2     NaN   NaN  Soumyajit Majumdar  Sayani Gupta   Plabita Borthakur   
3     4.4    35          Ovais Khan       Prateik          Ishita Raj   
4     NaN   NaN        Amol Palekar  Rajat Kapoor  Rituparna Sengupta   

           Actor 3  
0  Rajendra Bhatia  
1    Arvind Jangid  
2       Roy Angana  
3  Siddhant Kapoor  
4    

In [5]:
df.columns

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')

In [7]:


missing_values = df.isnull()

missing_counts = df.isnull().sum()

print("Counts of missing values:")
print(missing_counts)


Counts of missing values:
Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64


In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler



# Handling missing values
# Identify columns with missing values
print(df.isnull().sum())

# Impute missing values for numerical columns
numerical_columns = ['Year', 'Duration', 'Rating', 'Votes']
imputer_numeric = SimpleImputer(strategy='mean')
df[numerical_columns] = imputer_numeric.fit_transform(df[numerical_columns])

# Impute missing values for categorical columns
categorical_columns = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
imputer_categorical = SimpleImputer(strategy='most_frequent')
df[categorical_columns] = imputer_categorical.fit_transform(df[categorical_columns])

# Data cleaning
# Example: Remove irrelevant columns
df.drop(columns=['Irrelevant_Column'], inplace=True)

# Feature engineering
# Example: Create new features, normalize numerical features
# Example: df['Year'] = pd.to_datetime(df['Date']).dt.year

# Handling categorical variables
# Example: One-hot encoding
df = pd.get_dummies(df, columns=['Genre'])

# Handling outliers (example of clipping outliers)
# Example: Define limits based on Z-score or IQR and clip values
# Example: df['Amount'] = df['Amount'].clip(lower=df['Amount'].quantile(0.05), upper=df['Amount'].quantile(0.95))

# Data integration, transformation, and validation steps would follow...

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X = df.drop('Target_Column', axis=1)
y = df['Target_Column']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further preprocessing or modeling steps as needed...


Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '(2019)'